In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, OPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [80]:
df = pd.read_csv(
    'ldw-base_relacionamento_atual.csv',
    delimiter=';',
    dtype={'cod_carteira': str}
    )\
    .astype(
        {
            'cod_central': str,
            'cod_coop': str,
            'num_conta_principal': str,
            'cod_ua': str,
            'num_cpf_cnpj': str,
            'cod_cnae': str,
            'ano_mes': str,
            'flg_cheque_especial': str,
            'num_ano_mes': str,
            'flg_associado': str
        }
    )\
    .assign(
        cad_pix=lambda x: x.cad_pix.map({'S': '1', 'N': '0'}),
        assoc_desde=lambda x: pd.to_datetime(x.assoc_desde),
        tempo_assoc=lambda x: (pd.Timestamp('2024-11-01') - x.assoc_desde).dt.days,
        ultimo_contato=lambda x: pd.to_datetime(x.ultimo_contato),
        nivel_risco=lambda x: x.nivel_risco.fillna('DEFAULT'),
        score_principalidade=lambda x: x.score_principalidade.fillna(0),
        faixa_principalidade=lambda x: x.faixa_principalidade.replace('MISSING', np.nan).fillna('BAIXA'),
        publico_estrategico=lambda x: x.publico_estrategico.fillna('cooperativa'),
        qt_investimento_media=lambda x: x.loc[:, [
            'qt_fundos', 'qt_lci', 'qt_lca',
            'qt_dep_a_prazo']].mean(axis=1),
        alavancagem=lambda x: x.sld_cred_scr_2m / x.renda_mensal,
        flg_sexo=lambda x: [
            'PJ' if x.segmento.loc[i] == "PJ"
            else x.flg_sexo.loc[i]
            for i in range(x.shape[0])
        ]
    )\
    .query('(status_associado == "ATIVO") and (renda_mensal > 0)')\
    .drop(
        columns=[
            'sld_cred_scr_180_2m', 'sld_cred_sicredi_sas_2m',
            'sld_cred_sicredi_sas_180_2m', 'mc_seguro_agricola',
            'vlr_prej_coobrigacoes', 'vlr_prej_outros', 'sld_cred_moeda',
            'sld_seg_elementares', 'sld_seg_agricola', 'fat_ano',
            'mc_seg_elementares', 'mc_domicilio', 'mc_cred_moeda',
            'qt_cred_moeda', 'qt_seg_elementares', 'qt_seg_agricola'
        ]
    )\
    .dropna(subset=['idade'])\
    .reset_index(drop=True)

df = df\
    .astype(
        {
            col: str
            for col in df.loc[:,df.columns.str.startswith(
                ('prod', 'flg', 'digital',
                 'possui', 'ib', 'mobi', 'fone')
            )].columns
        }
    )\
    .assign(
        qt_investimento_media=lambda x: x.loc[:, [
            'qt_fundos', 'qt_lci', 'qt_lca',
            'qt_dep_a_prazo']].mean(axis=1)
    )

df.loc[(df.idade < 15) & (df.renda_mensal > 1e2), 'renda_mensal'] = 0
df.loc[
    (df.idade >= 19) &
    (df.renda_mensal <= 100) &
    (~df.desc_cbo.isin(['Estudantes'])),
    'renda_mensal'] = np.nan
df.loc[df.renda_mensal.isna(), 'renda_mensal'] = df\
    .groupby('desc_cbo')\
    .renda_mensal\
    .transform('median')

In [5]:
knn = KNNImputer(n_neighbors=10)
knn_cols = df.dropna(subset=['renda_mensal'])\
    .select_dtypes(include=np.number).corr(method='spearman')\
    .renda_mensal.abs().sort_values(ascending=False).head(50).index\
    .tolist()
df_impute_knn = knn.fit_transform(df[knn_cols])
df_impute_knn = pd.DataFrame(
    df_impute_knn,
    columns=knn_cols
    )
df['renda_mensal'] = df_impute_knn.renda_mensal

In [ ]:
df_cluster = df\
    .loc[:, [
        'idade', 'renda_mensal',
        'tempo_assoc', 'score_principalidade',
        'isa', 'cad_pix', 'digital_acessou_30d',
        'mobi_acessou_30d', 'sld_previdencia',
        'qt_investimento_media', 'alavancagem'] +
        df.columns[df.columns.str.startswith('sld_seg')].tolist() +
        df.columns[df.columns.str.startswith('sld_cred')].tolist()
    ]\
    .astype(
        {
            'digital_acessou_30d': int,
            'mobi_acessou_30d': int
        }
    )

df_cluster_columns = df_cluster.columns.tolist()

df_pca = df\
    .loc[:, 
         df.columns[df.columns.str.startswith('vlr_prov')].tolist() +
         df.columns[df.columns.str.startswith('vlr_prej')].tolist() +
         df.columns[df.columns.str.startswith('mc_')].tolist() +
         ['renda_mensal', 'idade', 'tempo_assoc']
    ]

In [9]:
scaler = StandardScaler()
pca = PCA(n_components=3)
segment_scaler = scaler.fit_transform(
    df_pca.select_dtypes(include=np.number)
    )
pca.fit(segment_scaler)
scores_pca = pca.transform(segment_scaler)

scale_PC1 = 1 / (scores_pca[:,0].max() - scores_pca[:,0].min())
scale_PC2 = 1 / (scores_pca[:,1].max() - scores_pca[:,1].min())
scale_PC3 = 1 / (scores_pca[:,2].max() - scores_pca[:,2].min())

features = df_pca.columns.tolist()
loadings = pca.components_

In [ ]:
class Inertia: 
    def __init__(self, data, cluster_columns=None, set_groups=20):
        self.set_groups = set_groups + 1
        self.data = data
        self.data_cluster = data.loc[:, cluster_columns]
        self.cluster_columns = cluster_columns

    def find_groups(self, n_clusters, init=''):
        return KMeans(
            n_clusters=n_clusters,
            init='k-means++',
            random_state=42
            ).fit(self.data_cluster)\
            .labels_
    
    def calc_inertia(self):
        return [
            KMeans(
                n_clusters=i,
                init='k-means++',
                random_state=42
            ).fit(self.data_cluster)\
            .labels_
            for i in range(1, self.set_groups)
        ]

    def plot_inertia(self):
        plt.figure(figsize=(10, 8))
        plt.plot(
            range(1, self.set_groups),
            self.calc_inertia(),
            marker='o', 
            linestyle='--'
        )

In [74]:
df = df.assign(
    Grupos=Inertia(df_cluster, df_cluster_columns).find_groups(6)
    )

In [79]:
df.query('Grupos == 1').loc[:, df_cluster_columns]

,idade,renda_mensal,tempo_assoc,score_principalidade,isa,cad_pix,digital_acessou_30d,mobi_acessou_30d,sld_previdencia,qt_investimento_media,...,sld_seg_residencial,sld_seg_prestamista,sld_seg_patrimonial,sld_seg_rural,sld_seg_out_familias,sld_cred_coobrigacoes,sld_cred_direcionados,sld_cred_rural,sld_cred_comercial,sld_cred_scr_2m
3525,68.0,7104.600,10411,0.294507,6,N,1,1,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,56933.12,799475.84
7883,76.0,7120.000,242,0.422499,3,S,1,1,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,372499.53
10321,62.0,7104.600,4775,0.145725,3,N,1,1,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,549280.69
12686,49.0,27810.405,7714,0.416167,5,N,1,1,0.00,1.50,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,675500.46
20151,53.0,16493.790,3509,0.262222,3,S,1,1,0.00,3.25,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,416834.99
20614,39.0,3132.625,2032,0.420000,5,S,1,1,0.00,0.50,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,381112.41
22344,30.0,8764.430,1717,0.566667,2,S,1,1,0.00,0.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,637617.80
22830,77.0,7104.600,4645,0.210311,2,N,0,0,0.00,2.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,550406.41
26538,40.0,13664.760,6724,0.347885,1,S,0,0,0.00,1.00,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,617979.31
27333,59.0,7104.600,2272,0.540828,5,S,0,0,0.00,0.25,...,0.00,0.24,0.00,0.0,0.0,0.0,0.0,0.00,563.59,499313.42


In [69]:
teste = Inertia(
    df.query('Grupos == 1'),
    df_cluster_columns
)

In [ ]:
#wcss = []
#for i in range(1, 21):
#    kmeans_pca = KMeans(
#        n_clusters=i,
#        init='k-means++',
#        random_state=42
#    )
#    kmeans_pca.fit(df_cluster)
#    wcss.append(kmeans_pca.inertia_)

In [47]:
#plt.figure(figsize=(10, 8))
#plt.plot(range(1, 21), wcss, marker='o', linestyle='--')

In [ ]:
kmeans_pca = KMeans(n_clusters=6, init='k-means++', random_state=42)
kmeans_pca.fit(df_cluster)

KMeans(n_clusters=6, random_state=42)

In [26]:
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(scores_pca)], axis=1)
df.columns.values[-3:] = ['PC1', 'PC2', 'PC3']
df['Grupos'] = kmeans_pca.labels_.astype(str)
df = df.sort_values('Grupos')\
    .assign(
        Grupos=lambda x: x.Grupos.map(
            {
                '0': 'cluster 1',
                '1': 'cluster 2',
                '2': 'cluster 3',
                '3': 'cluster 4',
                '4': 'cluster 5',
                '5': 'cluster 6'
            }
        )
    )

In [ ]:
for grupo in df.Grupos.unique()[0]:
    wcss = []
    for i in range(1, 21):
        kmeans = KMeans(
            n_clusters=i,
            init='k-means++',
            random_state=42)
        kmeans.fit(
            df\
                .query('Grupos == @grupo')\
                .loc[:, df_cluster.columns.tolist()]
        )
        wcss.append(kmeans.inertia_)

In [ ]:
#qtd_prods = df_segm_pca_kmeans\
#    .loc[:, 
#         ['Grupos'] +
#         df_segm_pca_kmeans.columns[
#             df_segm_pca_kmeans.columns.str.startswith('prod_')
#            ].tolist()
#         ]\
#    .set_index('Grupos')\
#    .stack()\
#    .reset_index(name='possui')\
#    .rename(columns={'level_1': 'produto'})\
#    .astype({'possui': int})

In [ ]:
#df_segm_pca_kmeans.to_csv('relacionamento_clusters.csv', index=False)